In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/My Drive/cs598ps-FP/FMA/

Mounted at /gdrive
/gdrive/My Drive/cs598ps-FP/FMA


In [ ]:
!pip install python-dotenv
import os
import sys
import glob
import torch
import numpy as np
import pickle

import IPython.display as ipd
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as skl
import sklearn.utils, sklearn.preprocessing, sklearn.decomposition, sklearn.svm
import librosa
import librosa.display

import utils

print(torch.cuda.is_available())

False


In [ ]:
# Load metadata and features.
tracks = utils.load('fma_metadata/tracks.csv')
genres = utils.load('fma_metadata/genres.csv')
features = utils.load('fma_metadata/features.csv')
echonest = utils.load('fma_metadata/echonest.csv')

np.testing.assert_array_equal(features.index, tracks.index)
assert echonest.index.isin(tracks.index).all()

print(tracks.shape, genres.shape, features.shape, echonest.shape)
small = tracks[tracks['set', 'subset'] <= 'small']
small.shape

(106574, 52) (163, 4) (106574, 518) (13129, 249)


(8000, 52)

In [ ]:
# Directory where mp3 are stored.
AUDIO_DIR = '/gdrive/My Drive/cs598ps-FP/FMA/fma_small'
# checkpoint dir
checkpoint_dir = '/gdrive/My Drive/cs598ps-FP/FMA/checkpoint'

In [ ]:
training_x = pickle.load(open(checkpoint_dir + '/training_x', "rb"))
training_y = pickle.load(open(checkpoint_dir + '/training_y', "rb"))
test_x = pickle.load(open(checkpoint_dir + '/test_x', "rb"))
test_y = pickle.load(open(checkpoint_dir + '/test_y', "rb"))
validation_x = pickle.load(open(checkpoint_dir + '/validation_x', "rb"))
validation_y = pickle.load(open(checkpoint_dir + '/validation_y', "rb"))
prev_cnt = len(training_x) + len(test_x) + len(validation_x)
print(prev_cnt)


7897


In [ ]:
# algorithm to extraction audio features from mp3 files in fma_small dataset
cnt = 0
for root, dirs, files in os.walk(AUDIO_DIR):
  for f in files:
    if '.mp3' in f:
      cnt += 1
      if cnt <= prev_cnt: continue
      filename = os.path.join(root, f)
      if os.stat(filename).st_size == 0: continue
      track_id = f[:-4]
      genre = tracks.loc[int(track_id), ('track', 'genre_top')]
      split = tracks.loc[int(track_id), ('set', 'split')]
      x, sr = librosa.load(filename, duration=29.95, sr=22050, mono=True)
      stft = np.abs(librosa.stft(x))**2
      mel = librosa.feature.melspectrogram(S=stft, sr=sr, n_mels=128, fmax=8000)
      mfcc = librosa.feature.mfcc(S=librosa.power_to_db(mel), n_mfcc=20)
      mfcc = skl.preprocessing.StandardScaler().fit_transform(mfcc)
      
      if split == 'training':
        training_x.append(mfcc)
        training_y.append(genre)
      elif split == 'test':
        test_x.append(mfcc)
        test_y.append(genre)
      else:
        validation_x.append(mfcc)
        validation_y.append(genre)

      if cnt % 100 == 0:
        pickle.dump(training_x, open(checkpoint_dir + '/training_x', 'wb'))
        pickle.dump(training_y, open(checkpoint_dir + '/training_y', 'wb'))
        pickle.dump(test_x, open(checkpoint_dir + '/test_x', 'wb'))
        pickle.dump(test_y, open(checkpoint_dir + '/test_y', 'wb'))
        pickle.dump(validation_x, open(checkpoint_dir + '/validation_x', 'wb'))
        pickle.dump(validation_y, open(checkpoint_dir + '/validation_y', 'wb'))
        print(cnt)

In [ ]:
genres = {}
for i in training_y:
  if i in genres:
    genres[i] += 1
  else:
    genres[i] = 1

In [ ]:
print(genres)

{'International': 788, 'Hip-Hop': 788, 'Folk': 796, 'Pop': 781, 'Experimental': 793, 'Rock': 797, 'Electronic': 783, 'Instrumental': 784}


In [ ]:
print(len(validation_x), len(training_x), len(test_x))

800 6310 787
